# Lab 10

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Thomatous/geog510-labs/blob/main/lab_10.ipynb)

In [ ]:
%pip install -q -U "geemap[workshop]"

In [ ]:
import ee
import geemap

In [ ]:
# ee.Authenticate(force=True)

In [ ]:
geemap.ee_initialize()

## Overview

This lab introduces Google Earth Engine (GEE) for accessing and analyzing geospatial data in Python. You will explore diverse data types, including DEM, satellite imagery, and land cover datasets. You’ll gain experience creating cloud-free composites, visualizing temporal changes, and generating animations for time-series data.


## Objectives

By completing this lab, you will be able to:

1. Access and visualize Digital Elevation Model (DEM) data for a specific region.
2. Generate cloud-free composites from Sentinel-2 or Landsat imagery for a specified year.
3. Visualize National Agricultural Imagery Program (NAIP) data for U.S. counties.
4. Display watershed boundaries using Earth Engine styling.
5. Visualize land cover changes over time using split-panel maps.
6. Create a time-lapse animation for land cover changes over time in a region of your choice.

## Exercise 1: Visualizing DEM Data

Find a DEM dataset in the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets) and clip it to a specific area (e.g., your country, state, or city). Display it with an appropriate color palette. For example, the sample map below shows the DEM of the state of Colorado.

![](https://i.imgur.com/OLeSt7n.png)

In [ ]:
from collections import namedtuple
Region = namedtuple('Region', ['bbox', 'highest_point', 'center'])

peloponnese = Region(
    bbox=ee.Geometry.BBox(20.315, 36.08, 24.267,  38.526),
    highest_point=2405,
    center=[37.303, 22.291],
)

In [ ]:
m_elevation = geemap.Map(center=peloponnese.center, zoom=8, height="600px")

dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select('elevation')
elevation_clipped = elevation.clip(peloponnese.bbox)

outline = ee.FeatureCollection(ee.Feature(peloponnese.bbox))
outline_styled = outline.style(
    color='FF0000',       
    width=2,              
    fillColor='00000000'
)

m_elevation.addLayer(elevation_clipped, {'min': 0, 'max': peloponnese.highest_point, 'palette': 'terrain'}, 'Elevation')
m_elevation.addLayer(outline_styled, {}, 'Boundary')
m_elevation

## Exercise 2: Cloud-Free Composite with Sentinel-2 or Landsat

Use Sentinel-2 or Landsat-9 data to create a cloud-free composite for a specific year in a region of your choice.

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) or [Landsat-9 data](https://developers.google.com/earth-engine/datasets/catalog/landsat-9) data to create a cloud-free composite for a specific year in a region of your choice. Display the imagery on the map with a proper band combination. For example, the sample map below shows a cloud-free false-color composite of Sentinel-2 imagery of the year 2021 for the state of Colorado. 

![](https://i.imgur.com/xkxpkS1.png)

In [ ]:
# NOTE: https://developers.google.com/earth-engine/guides/landsat#simple-composite

collection = ee.ImageCollection('LANDSAT/LT05/C02/T1').filterDate(
    '2010-01-01', '2010-12-31'
)

composite = ee.Algorithms.Landsat.simpleComposite(collection)
composite_clipped = composite.clip(peloponnese.bbox)

outline = ee.FeatureCollection(ee.Feature(peloponnese.bbox))
outline_styled = outline.style(
    color='000000',       
    width=2,              
    fillColor='00000000'
)

m_landsat = geemap.Map(center=peloponnese.center, zoom=8, height="600px")
m_landsat.add_layer(
    composite_clipped, {'bands': ['B4', 'B3', 'B2'], 'max': 128}, 'Landsat'
)
m_landsat.addLayer(outline_styled, {}, 'Boundary')
m_landsat


## Exercise 3: Visualizing NAIP Imagery

Use [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) imagery to create a cloud-free imagery for a U.S. county of your choice. For example, the sample map below shows a cloud-free true-color composite of NAIP imagery for Knox County, Tennessee. Keep in mind that there might be some counties with the same name in different states, so make sure to select the correct county for the selected state.

![](https://i.imgur.com/iZSGqGS.png)

In [ ]:
counties = ee.FeatureCollection('TIGER/2018/Counties')
county = counties.filter(
    ee.Filter.And(
        ee.Filter.eq('NAME', 'Inyo'),
        ee.Filter.eq('STATEFP', '06')
    )
)

naip = ee.ImageCollection('USDA/NAIP/DOQQ').filterBounds(county.geometry()).filterDate('2018-01-01', '2022-12-31')
clipped_naip = naip.median().clip(county.geometry())

m_naip = geemap.Map(center=[36.5, -117], zoom=7)
m_naip.addLayer(clipped_naip, { }, 'NAIP')

boundary = county.style(**{
    'color': 'FF0000',
    'width': 2,
    'fillColor': '00000000'
})
m_naip.addLayer(boundary, {}, 'Boundary')
m_naip


## Exercise 4: Visualizing Watershed Boundaries

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

![](https://i.imgur.com/PLlNFq3.png)

In [ ]:
basins = ee.FeatureCollection('USGS/WBD/2017/HUC04')

m = geemap.Map(center=[40, -100], zoom=4, height="600px")

style_params = {
    'color': '0000FF',       
    'width': 2,
    'fillColor': '00000000' 
}
styled = basins.style(**style_params)

m.add_layer(styled, {}, 'Basins')
m

## Exercise 5: Visualizing Land Cover Change

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

![](https://i.imgur.com/Au7Q5Ln.png)

In [ ]:
from geemap.legends import builtin_legends

states = ee.FeatureCollection("TIGER/2018/States")
state = states.filter(ee.Filter.eq("NAME", "Minnesota"))

nlcd_2001 = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2001").select("landcover").clip(state.geometry())
nlcd_2019 = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select("landcover").clip(state.geometry())

left_layer = geemap.ee_tile_layer(nlcd_2001, {}, "NLCD 2001")
right_layer = geemap.ee_tile_layer(nlcd_2019, {}, "NLCD 2019")

m = geemap.Map()

left_layer = geemap.ee_tile_layer(nlcd_2001, {}, "NLCD 2001")
right_layer = geemap.ee_tile_layer(nlcd_2019, {}, "NLCD 2019")

m.split_map(left_layer, right_layer)
m.center_object(state, 7)
m.add_legend(builtin_legend="NLCD", max_width="100px", height="455px")
m

## Exercise 6: Creating a Landsat Timelapse Animation

Generate a timelapse animation using Landsat data to show changes over time for a selected region.

![Spain](https://github.com/user-attachments/assets/f12839c0-1c30-404d-b0ab-0fa12ce12d24)

In [ ]:
roi = ee.Geometry.BBox(742.643509, 39.393224, 743.010864, 39.597488)
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="lake_karla.gif",
    start_year=1984,
    end_year=2023,
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=3,
    title="Restoration of lake Karla, Thessaly, Greece",
    font_color="#202020",
)
geemap.show_image(timelapse)
